In [12]:
import pandas as pd
import numpy as np

# Energy

In [13]:
# # https://dacon.io/competitions/official/235720/codeshare/2583?page=1&dtype=recent 참고

# load
data1 = pd.read_csv('한국동서발전 시간대별 태양광 및 풍력 발전량 현황(2018_2019).csv', encoding='euc-kr')
data2 = pd.read_csv('한국동서발전 시간대별 태양광 발전량 현황(2015_2017).csv', encoding='euc-kr')

# convert name
convert_name = {'당진태양광':'dangjin', '당진수상태양광':'dangjin_floating', '당진자재창고태양광':'dangjin_warehouse', '울산태양광':'ulsan'}

In [14]:
def preprocess(energy_type):
    # sample
    data1_ = data1.loc[data1['태양광명'] == energy_type]
    data2_ = data2.loc[data2['태양광명'] == energy_type]

    # merge two data
    data1_.drop(columns=['태양광명'], inplace=True)
    data2_.drop(columns=['태양광명','용량(MW)'], inplace=True)
    data = data2_.append(data1_)

    # datetime
    data['시간'] = pd.to_datetime(data['시간'])

    # modify df
    hours = [] # to store 'time'
    values = [] # to store energy value

    for i in range(data.shape[0]):
        row = data.iloc[i]
        day = row['시간']

        for h in range(24): # loop over one row(one day)
            hour = day.replace(hour=h)
            hour += pd.Timedelta(1, unit='hour') # data is originaly 1~24. Change to 0~23.

            hours.append(hour)
            values.append(row[str(h+1)])

    df = pd.DataFrame({'time':hours, convert_name[energy_type]:values})
    return df.set_index('time')

In [15]:
dangjin = preprocess('당진태양광')
dangjin_floating = preprocess('당진수상태양광')
dangjin_warehouse = preprocess('당진자재창고태양광')
ulsan = preprocess('울산태양광')

/home/yejoon/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [16]:
# shape
print(dangjin.shape)
print(dangjin_floating.shape)
print(dangjin_warehouse.shape)
print(ulsan.shape)

(40824, 1)
(14592, 1)
(40776, 1)
(39408, 1)


In [17]:
# merge
dangjin_all = pd.concat([dangjin,dangjin_floating,dangjin_warehouse], axis=1, join='outer')

# slice time 
dangjin_all = dangjin_all.loc[dangjin_all.index < pd.Timestamp('2018-03-02 00:00:00')]
ulsan = ulsan.loc[ulsan.index < pd.Timestamp('2018-03-02 00:00:00')]

## Fill na in dangjin_floating

In [18]:
print(dangjin_all.isna().sum())
print(ulsan.isna().sum())

dangjin                  0
dangjin_floating     26232
dangjin_warehouse        0
dtype: int64
ulsan    0
dtype: int64


In [19]:
dangjin_all_mean_series = dangjin_all.mean(axis=0)

djdjw_to_djflt = dangjin_all_mean_series['dangjin_floating']  / (dangjin_all_mean_series['dangjin'] + dangjin_all_mean_series['dangjin_warehouse'])

djdjw_to_djflt

0.48344334791995036

In [20]:
na_dangjin_floating = dangjin_all.loc[dangjin_all['dangjin_floating'].isna()] # df where dangjin_floating = na
na_dangjin_floating.loc[:,'dangjin_floating'] = (na_dangjin_floating['dangjin'] + na_dangjin_floating['dangjin_warehouse']) * djdjw_to_djflt

dangjin_all_filled = dangjin_all.copy()
dangjin_all_filled[dangjin_all_filled['dangjin_floating'].isna()] = na_dangjin_floating

/home/yejoon/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [21]:
dangjin_all_filled

,dangjin,dangjin_floating,dangjin_warehouse
time,,,
2015-01-01 01:00:00,0,0.0,0.0
2015-01-01 02:00:00,0,0.0,0.0
2015-01-01 03:00:00,0,0.0,0.0
2015-01-01 04:00:00,0,0.0,0.0
2015-01-01 05:00:00,0,0.0,0.0
...,...,...,...
2018-03-01 19:00:00,0,11.0,2.0
2018-03-01 20:00:00,0,0.0,0.0
2018-03-01 21:00:00,0,0.0,0.0


# Fcst

In [30]:
from glob import glob
dj_path = sorted(glob('./dangjin_*/*.csv'))
dj_path

['./dangjin_2015/석문면_3시간기온_201501_201512.csv',
 './dangjin_2015/석문면_습도_201501_201512.csv',
 './dangjin_2015/석문면_풍속_201501_201512.csv',
 './dangjin_2015/석문면_풍향_201501_201512.csv',
 './dangjin_2015/석문면_하늘상태_201501_201512.csv',
 './dangjin_2016/석문면_3시간기온_201601_201612.csv',
 './dangjin_2016/석문면_습도_201601_201612.csv',
 './dangjin_2016/석문면_풍속_201601_201612.csv',
 './dangjin_2016/석문면_풍향_201601_201612.csv',
 './dangjin_2016/석문면_하늘상태_201601_201612.csv',
 './dangjin_2017/석문면_3시간기온_201701_201712.csv',
 './dangjin_2017/석문면_습도_201701_201712.csv',
 './dangjin_2017/석문면_풍속_201701_201712.csv',
 './dangjin_2017/석문면_풍향_201701_201712.csv',
 './dangjin_2017/석문면_하늘상태_201701_201712.csv',
 './dangjin_2018/석문면_3시간기온_201801_201802.csv',
 './dangjin_2018/석문면_습도_201801_201802.csv',
 './dangjin_2018/석문면_풍속_201801_201802.csv',
 './dangjin_2018/석문면_풍향_201801_201802.csv',
 './dangjin_2018/석문면_하늘상태_201801_201802.csv']

In [68]:
data = pd.read_csv(dj_path[0], header=0, names=['day','hour','forecast','value'])

# use the most recent fcst (forget about data leakage)
data = data.loc[data['forecast'] ==4.0]

# to make datetime
data['hour'] = data['hour'].apply(lambda x: int(x//100))
data['forecast'] = data['forecast'].astype('int')
data['day'] = data['day'].applly(lambda x:)

,day,hour,forecast,value
0,1,2,4,-2.4
20,1,5,4,-3.0
39,1,8,4,-5.0
57,1,11,4,-5.9
74,1,14,4,-5.7
...,...,...,...,...
52439,31,11,4,-1.3
52456,31,14,4,-2.0
52472,31,17,4,-2.2
52487,31,20,4,1.0


In [81]:
data['day'].unique()

array([' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9', ' 10', ' 11',
       ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19', ' 20',
       ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27', ' 28', ' 29',
       ' 30', ' 31'], dtype=object)

In [83]:
data.loc[data['day'] == ' 1']

,day,hour,forecast,value
0,1,2,4,-2.4
20,1,5,4,-3.0
39,1,8,4,-5.0
57,1,11,4,-5.9
74,1,14,4,-5.7
...,...,...,...,...
47999,1,11,4,5.9
48016,1,14,4,5.7
48032,1,17,4,6.7
48047,1,20,4,9.3
